<a href="https://colab.research.google.com/github/coughlinjennie/data71200/blob/main/projects/DATA71200_Project2_Coughlin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Supervised Learning
Because the field I want to use for labels is categorical — the property type — I'm using classifier models for this portion of the project. SVM, Gaussian naive Bayes, decision tree and KNN are the ones I'm considering.  


In [91]:
#Import the libraries and install scikit-learn
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import requests
import io
import matplotlib.pyplot as plt


!pip install -U scikit-learn==1.4

#Step 1: Import, split and clean the data
This is brought over from Project 1, with a fix to stratify when I split the data and a data pipeline for cleaning the data now that I know what needs to be done.

In [107]:
#Import the data, sourced from Kaggle and stored in my GitHub
url = "https://raw.githubusercontent.com/coughlinjennie/data71200/main/projects/nyhousing.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
#Load the data

housing_master = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [108]:
housing_master["TYPE"].value_counts()

TYPE
Co-op for sale                1450
House for sale                1012
Condo for sale                 891
Multi-family home for sale     727
Townhouse for sale             299
Pending                        243
Contingent                      88
Land for sale                   49
For sale                        20
Foreclosure                     14
Condop for sale                  5
Coming Soon                      2
Mobile house for sale            1
Name: count, dtype: int64

I need to stratify the data when I split it, and the two values in this field that will interfere with that are ones I was going to drop anyway because they're not relevant for this model. (The TYPE field is showing the status of the property, but I'm using only the labels that indicate the property type and exclude the others, plus a couple that aren't super-relevant in New York.) We're not supposed to clean data until after we split it, but I can't figure out how to stratify the data for the split without doing this one step, so I'm going to do it anyway.

In [109]:
# Delete all rows where column 'TYPE' has certain values
indexType = housing_master[ (housing_master['TYPE'] == "For sale") | (housing_master['TYPE'] == "Contingent") | (housing_master['TYPE'] == "Land for sale") | (housing_master['TYPE'] == "Foreclosure") | (housing_master['TYPE'] == "Pending") | (housing_master['TYPE'] == "Coming Soon") | (housing_master['TYPE'] == "Mobile house for sale") ].index
housing_master.drop(indexType , inplace=True)

In [110]:
housing_master["TYPE"].value_counts()

TYPE
Co-op for sale                1450
House for sale                1012
Condo for sale                 891
Multi-family home for sale     727
Townhouse for sale             299
Condop for sale                  5
Name: count, dtype: int64

In [111]:
#Set the labels on TYPE

housing_label = housing_master["TYPE"]

#Set the data
housing = housing_master.drop("TYPE", axis=1)
print(housing)

                                            BROKERTITLE      PRICE  BEDS  \
0           Brokered by Douglas Elliman  -111 Fifth Ave     315000     2   
1                                   Brokered by Serhant  195000000     7   
2                                Brokered by Sowae Corp     260000     4   
3                                   Brokered by COMPASS      69000     3   
4     Brokered by Sotheby's International Realty - E...   55000000     7   
...                                                 ...        ...   ...   
4796                                Brokered by COMPASS     599000     1   
4797                    Brokered by Mjr Real Estate Llc     245000     1   
4798      Brokered by Douglas Elliman - 575 Madison Ave    1275000     1   
4799            Brokered by E Realty International Corp     598125     2   
4800                 Brokered by Nyc Realty Brokers Llc     349000     1   

           BATH  PROPERTYSQFT  \
0      2.000000   1400.000000   
1     10.000000  1754

In [112]:
#Divide the data into training and testing sets
from sklearn.model_selection import train_test_split

housing_train, housing_test, housing_label_train, housing_label_test = train_test_split(housing, housing_label, test_size=0.3, stratify=housing_label, random_state=42)


Now that the data is split, I'm going to do some cleaning before the pre-processing. Some of these steps will only apply to the training set, including removing some extreme outliers. Others, like creating a ZIP code column based of a portion of an address field, will be done on both so the column exists in the testing data.

In [113]:
#Create a column with the ZIP code of the property for both training and testing data
housing_train["ZIP"] = housing_train.MAIN_ADDRESS.str[-5:]
housing_test["ZIP"] = housing_test.MAIN_ADDRESS.str[-5:]

In [114]:
# Create a list of redundant column names to drop from the training data only
to_drop = ["LONGITUDE", "LATITUDE", "ADDRESS", "ADMINISTRATIVE_AREA_LEVEL_2", "LOCALITY", "SUBLOCALITY", "FORMATTED_ADDRESS", "MAIN_ADDRESS", "STATE", "STREET_NAME","LONG_NAME","BROKERTITLE"]

# Drop those columns from the dataset
housing_subset = housing_train.drop(to_drop, axis = 1)



The dropped columns are redundant, but if they exist in the testing data the model just won't use them. So this step was only done on the training data.

In [115]:
#Drop all properties that sold for more than $1B from training data only

housing_clean = housing_subset[housing_subset['PRICE'] <= 100000000]
label_train_clean = housing_label_train[housing_subset['PRICE'] <= 100000000]


In [116]:
housing_clean.info()
label_train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3067 entries, 2589 to 2311
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PRICE         3067 non-null   int64  
 1   BEDS          3067 non-null   int64  
 2   BATH          3067 non-null   float64
 3   PROPERTYSQFT  3067 non-null   float64
 4   ZIP           3067 non-null   object 
dtypes: float64(2), int64(2), object(1)
memory usage: 143.8+ KB
<class 'pandas.core.series.Series'>
Index: 3067 entries, 2589 to 2311
Series name: TYPE
Non-Null Count  Dtype 
--------------  ----- 
3067 non-null   object
dtypes: object(1)
memory usage: 47.9+ KB


In [117]:
housing_clean.describe()

,PRICE,BEDS,BATH,PROPERTYSQFT
count,3.067000e+03,3067.000000,3067.000000,3067.000000
mean,1.955427e+06,3.401043,2.392915,2202.658046
std,4.342835e+06,2.810770,2.058869,2420.874714
min,4.950000e+04,1.000000,0.000000,250.000000
25%,4.990000e+05,2.000000,1.000000,1166.500000
50%,8.450000e+05,3.000000,2.000000,2184.207862
75%,1.499000e+06,4.000000,3.000000,2184.207862
max,6.500000e+07,50.000000,50.000000,65535.000000


In [120]:
housing_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3067 entries, 2589 to 2311
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PRICE         3067 non-null   int64  
 1   BEDS          3067 non-null   int64  
 2   BATH          3067 non-null   float64
 3   PROPERTYSQFT  3067 non-null   float64
 4   ZIP           3067 non-null   object 
dtypes: float64(2), int64(2), object(1)
memory usage: 143.8+ KB


#Step 2: Prepare the Data
Once the data is cleaned, I need to process it so I can run various supervised models on it.

Before I run the pipeline, I'm setting up column names for the encoded ZIP code data.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

zip_cat = [("ZIP", ["10453","10457","10460","10458","10467","10468","10451","10452","10456","10454","10455","10459","10474","10463","10471","10466","10469","10470","10475","10461","10462","10464","10465","10472","10473","11212","11213","11216","11233","11238","11209","11214","11228","11204","11218","11219","11230","11234","11236","11239","11223","11224","11229","11235","11201","11205","11215","11217","11231","11203","11210","11225","11226","11207","11208","11211","11222","11220","11232","11206","11221","11237","10026","10027","10030","10037","10039","10001","10011","10018","10019","10020","10036","10029","10035","10010","10016","10017","10022","10012","10013","10014","10004","10005","10006","10007","10038","10280","10002","10003","10009","10021","10028","10044","10065","10075","10128","10023","10024","10025","10031","10032","10033","10034","10040","11361","11362","11363","11364","11354","11355","11356","11357","11358","11359","11360","11365","11366","11367","11412","11423","11432","11433","11434","11435","11436","11101","11102","11103","11104","11105","11106","11374","11375","11379","11385","11691","11692","11693","11694","11695","11697","11004","11005","11411","11413","11422","11426","11427","11428","11429","11414","11415","11416","11417","11418","11419","11420","11421","11368","11369","11370","11372","11373","11377","11378","10302","10303","10310","10306","10307","10308","10309","10312","10301","10304","10305","10314"] )]




enc = OneHotEncoder()

transformer = make_column_transformer((enc, ohe_columns), remainder='passthrough')

# Convert the resulting arrays to DataFrames
transformed=transformer.fit_transform(housing_clean)
display(pd.DataFrame(
    transformed,
    columns=transformer.get_feature_names_out(),
    index=housing_clean.index
))
pd.DataFrame(
    transformer.transform(housing_test),
    columns=transformer.get_feature_names_out(),
    index=housing_test.index
)



In [133]:

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

ohe = ColumnTransformer(
    transformers = [("ohe", encoder, ["ZIP"]),],
    remainder = "passthrough",
)
ohe.set_output(transform="pandas")

ohe.fit(housing_clean)
h_enc_train = ohe.transform(housing_clean)
h_enc_test = ohe.transform(housing_test)
h_enc_train.head()

,ohe__ZIP_10001,ohe__ZIP_10002,ohe__ZIP_10003,ohe__ZIP_10004,ohe__ZIP_10005,ohe__ZIP_10006,ohe__ZIP_10007,ohe__ZIP_10009,ohe__ZIP_10010,ohe__ZIP_10011,...,ohe__ZIP_11436,ohe__ZIP_11691,ohe__ZIP_11692,ohe__ZIP_11693,ohe__ZIP_11694,ohe__ZIP_11697,remainder__PRICE,remainder__BEDS,remainder__BATH,remainder__PROPERTYSQFT
2589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,749000,3,2.0,1800.000000
4269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,449000,1,2.0,1074.000000
2244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1999999,3,2.0,1865.000000
1352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,979000,3,2.0,1272.000000
1401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,329000,1,1.0,2184.207862


In [123]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

housing_zip = housing_clean["ZIP"].values.reshape(-1,1)
cat_encoder = OneHotEncoder()
housing_zip_1hot = cat_encoder.fit_transform(housing_zip)
housing_zip_1hot

<3067x172 sparse matrix of type '<class 'numpy.float64'>'
	with 3067 stored elements in Compressed Sparse Row format>

In [124]:
housing_zip_1hot.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [118]:
#Organize columns into scaled numeric, unscaled numeric and categorical

sc_housing = "BEDS", "BATH", "PROPERTYSQFT"
unsc_housing = "PRICE"
cat_housing = "ZIP"

In [ ]:
#Import pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

#Set up numeric and categorical pipelines
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),

])

#  ("transform", FunctionTransformer(np.log, inverse_func = np.exp)

cat_pipeline = Pipeline([
    ("impute_c", SimpleImputer(strategy = "most_frequent")),
    ("encode", OneHotEncoder(handle_unknown="ignore"))
])

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_housing),
    ("cat", cat_pipeline,cat_housing),
])

In [ ]:
#Prepare the data
housing_prepared = preprocessing.fit_transform(housing_clean)
housing_prepared.shape
#housing_final = pd.DataFrame(housing_prepared, columns=housing.columns)

#housing_final_df.head()

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

The first few times I ran this, there were a lot more columns. I ended up dropping some of the categorical columns that gave location information to streamline things, which took it from 4K columns and change down to 1156. Still a ton, but one hot encoding for ZIP codes meant there were always going to be a bunch.

When I first tried running the classifier models, I got an error for infinite values, so I added checks after both the cleaning and the pre-processing.

#Step 3: Examine the Target Attribute
The TYPE field is my target attribute

In [106]:
#Examine the distribution of the categories

type_group = label_train_clean.groupby("TYPE").size()
type_group.plt.bar()

KeyError: 'TYPE'

#Step 4: Select Classifier Models

Since I'm trying to predict a categorical label — the type of property — I'm only assessing classifier models for this project. I'm going to try K-Nearest Neighbors and the Decision Tree classifiers

In [ ]:
#import models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection



#Step 5: Run and Assess the Models
Three components here:

1.   Run with the defaults
2.   Run again and use cross-validation
3.   Adjust parameters for the model(s) using grid search




In [ ]:
#Run K-Neighbors with defaults
knn = KNeighborsClassifier()
knn.fit(housing_prepared, label_train_clean)
knn.predict()

In [ ]:

tree = DecisionTreeClassifier()
tree.fit(housing_prepared, label_train_clean)